# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
city_data = pd.read_csv("../WeatherPy/output_data/cities.csv")
city_data.head()


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country
0,hobart,-42.88,147.33,44.60,65,40,3.10,AU
1,bathsheba,13.22,-59.52,86.00,70,40,8.20,BB
2,berovo,41.70,22.86,74.97,42,0,2.97,MK
3,ikom,5.97,8.71,78.96,85,88,1.64,NG
4,kenai,60.55,-151.26,55.40,93,90,2.10,US


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)
locations = city_data[["Lat", "Lng"]]
humidity = city_data["Humidity"]

In [4]:
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, 
                                 max_intensity=100, point_radius=1)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
index = city_data[(city_data["Humidity"] > 50) | \
                  (city_data["Wind Speed"] > 5) | \
                  (city_data["Max Temp"] < 70) | \
                  (city_data["Max Temp"] > 80) | \
                  (city_data["Cloudiness"] > 20)].index.tolist()
hotel_df = city_data.drop(index)
hotel_df = hotel_df.dropna()
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country
2,berovo,41.70,22.86,74.97,42,0,2.97,MK
12,east london,-33.02,27.91,75.20,17,0,2.60,ZA
119,port alfred,-33.59,26.89,71.58,33,0,3.71,ZA
151,lishu,43.30,124.33,70.48,43,0,3.88,CN
223,ostrovnoy,68.05,39.51,75.49,45,17,2.63,RU
260,allonnes,47.97,0.16,77.00,50,7,4.60,FR
275,phalombe,-15.81,35.65,72.52,43,7,2.53,MW
370,maracaju,-21.61,-55.17,74.23,41,0,3.85,BR
385,yablonovo,50.85,37.36,77.00,33,2,1.59,RU
397,puerto suarez,-18.95,-57.80,78.80,50,0,4.10,BO


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df["Hotel Name"] = ""

In [7]:
target_search = "Hotel"
target_radius = 5000
target_type = "hotel"
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "keyword": target_search,
    "radius": target_radius,
    "type": target_type,
    "key": g_key
}
      
for index, row in hotel_df.iterrows():
    params["location"] = f"{row['Lat']},{row['Lng']}"
    response = requests.get(base_url, params=params).json()
    results = response["results"]
    try:
        hotel_df.loc[index, "Hotel Name"] = results[0]["name"]
    except (KeyError, IndexError):
        print(f"There is no hotel within 5 km of {row['City'].title()}.")

There is no hotel within 5 km of Ostrovnoy.
There is no hotel within 5 km of Phalombe.
There is no hotel within 5 km of Yablonovo.


In [8]:
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Hotel Name
2,berovo,41.70,22.86,74.97,42,0,2.97,MK,Aurora Resort & Spa
12,east london,-33.02,27.91,75.20,17,0,2.60,ZA,Premier Hotels & Resorts
119,port alfred,-33.59,26.89,71.58,33,0,3.71,ZA,The Halyards Hotel
151,lishu,43.30,124.33,70.48,43,0,3.88,CN,梨树韩州宾馆
223,ostrovnoy,68.05,39.51,75.49,45,17,2.63,RU,
260,allonnes,47.97,0.16,77.00,50,7,4.60,FR,Hôtel Mercure Le Mans Centre
275,phalombe,-15.81,35.65,72.52,43,7,2.53,MW,
370,maracaju,-21.61,-55.17,74.23,41,0,3.85,BR,Hotel Pousada da Serra
385,yablonovo,50.85,37.36,77.00,33,2,1.59,RU,
397,puerto suarez,-18.95,-57.80,78.80,50,0,4.10,BO,Hotel casa real


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [12]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))